In [2]:
import logging
import wandb
import joblib
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

WANDB_API_KEY=os.environ.get('WANDB_API_KEY')

In [4]:
# Login to Weights & Biases
!wandb login --relogin $WANDB_API_KEY

wandb: WARNING Using legacy-service, which is deprecated. If this is unintentional, you can fix it by ensuring you do not call `wandb.require('legacy-service')` and do not set the WANDB_X_REQUIRE_LEGACY_SERVICE environment variable.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\pv\_netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [14]:
# ratio used to split train and validation data
val_size = 0.2
# seed used to reproduce purposes
seed = 42
# reference (column) to stratify the data
stratify = "512"
# name of the input artifact
artifact_input_name = "attendance_face_recognition/train.csv:latest"
# type of the artifact
artifact_type = "Train"

In [15]:
# configure logging
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(message)s",
                    datefmt='%d-%m-%Y %H:%M:%S')

# reference for a logging obj
logger = logging.getLogger()

# initiate the wandb project
run = wandb.init(project="attendance_face_recognition",job_type="train")

logger.info("Downloading and reading train artifact")
artifact = run.use_artifact(artifact_input_name)
artifact_dir = artifact.download()  # Lưu toàn bộ files về local

df_train = pd.read_csv(os.path.join(artifact_dir, "train.csv"))

# Spliting train.csv into train and validation dataset
logger.info("Spliting data into train/val")
# split-out train/validation and test dataset
x_train, x_val, y_train, y_val = train_test_split(df_train.drop(labels=stratify,axis=1),
                                                  df_train[stratify],
                                                  test_size=val_size,
                                                  random_state=seed,
                                                  shuffle=True,
                                                  stratify=df_train[stratify])

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


29-04-2025 23:10:22 Downloading and reading train artifact
wandb:   1 of 1 files downloaded.  
29-04-2025 23:10:25 Spliting data into train/val


In [6]:
logger.info("x train: {}".format(x_train.shape))
logger.info("y train: {}".format(y_train.shape))
logger.info("x val: {}".format(x_val.shape))
logger.info("y val: {}".format(y_val.shape))

29-04-2025 21:49:29 x train: (388, 512)
29-04-2025 21:49:29 y train: (388,)
29-04-2025 21:49:29 x val: (98, 512)
29-04-2025 21:49:29 y val: (98,)


In [7]:
sweep_config = {
    "method": "random",  # random search
    "metric": {
        "name": "val_accuracy",
        "goal": "maximize"
    },
    "parameters": {
        "n_estimators": {"values": [50, 100, 200, 300]},
        "max_depth": {"values": [5, 10, 20, None]},
        "min_samples_split": {"values": [2, 5, 10]},
        "min_samples_leaf": {"values": [1, 2, 5]},
        "max_features": {"values": ["sqrt", "log2", None]},
        "bootstrap": {"values": [True, False]}
    }
}

sweep_id = wandb.sweep(sweep_config, project="attendance_face_recognition")

Create sweep with ID: epiwaxnc
Sweep URL: https://wandb.ai/thanvinh164-vinh/attendance_face_recognition/sweeps/epiwaxnc


In [8]:
def train():
    # Mỗi lần agent chạy, sẽ tạo 1 run mới với các config khác nhau
    with wandb.init() as run:
        config = run.config
        
        # Xây dựng và train model với config hiện tại
        model = RandomForestClassifier(
            n_estimators=config.n_estimators,
            max_depth=config.max_depth,
            min_samples_split=config.min_samples_split,
            min_samples_leaf=config.min_samples_leaf,
            max_features=config.max_features,
            bootstrap=config.bootstrap,
            random_state=42,
            n_jobs=-1
        )
        model.fit(x_train, y_train)
        
        # Đánh giá trên validation set
        preds = model.predict(x_val)
        acc = accuracy_score(y_val, preds)
        
        # Log metrics lên W&B
        wandb.log({"validation accuracy": acc})

In [9]:
wandb.agent(sweep_id, function=train, count=200)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: hsloeeq7 with config:
wandb: 	bootstrap: False
wandb: 	max_depth: 5
wandb: 	max_features: None
wandb: 	min_samples_leaf: 1
wandb: 	min_samples_split: 5
wandb: 	n_estimators: 200


validation accuracy,▁
validation accuracy,0.96939


wandb: Agent Starting Run: 9ez5gczo with config:
wandb: 	bootstrap: True
wandb: 	max_depth: 10
wandb: 	max_features: log2
wandb: 	min_samples_leaf: 2
wandb: 	min_samples_split: 2
wandb: 	n_estimators: 300


validation accuracy,▁
validation accuracy,0.97959


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 85swd8pj with config:
wandb: 	bootstrap: True
wandb: 	max_depth: 20
wandb: 	max_features: log2
wandb: 	min_samples_leaf: 5
wandb: 	min_samples_split: 10
wandb: 	n_estimators: 50


...

validation accuracy,▁
validation accuracy,0.96939


wandb: Agent Starting Run: cgw1w9tl with config:
wandb: 	bootstrap: False
wandb: 	max_depth: 5
wandb: 	max_features: sqrt
wandb: 	min_samples_leaf: 1
wandb: 	min_samples_split: 2
wandb: 	n_estimators: 200


validation accuracy,▁
validation accuracy,0.96939


Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x000002AE41316F50>> (for post_run_cell), with arguments args (<ExecutionResult object at 2ae4133b910, execution_count=9 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 2ae4133a2c0, raw_cell="wandb.agent(sweep_id, function=train, count=200)" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/d%3A/Documents/Text%20Book/3/ML%20Ops/Final%20Project/Attendence_Checking_by_Facial_Recognition/3.%20Training_model.ipynb#X11sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine

In [9]:
# 1. Lấy config tốt nhất từ W&B
ENTITY   = "thanvinh164-vinh"       # thường là username hoặc team
PROJECT  = "attendance_face_recognition"
SWEEP_ID = "epiwaxnc"

In [10]:
api = wandb.Api()

runs = api.runs(f"{ENTITY}/{PROJECT}", {"sweep": SWEEP_ID})
best_run = max(runs, key=lambda r: r.summary.get("validation accuracy", 0.0))
best_cfg = best_run.config

print("Best hyperparameters:", best_cfg)

Best hyperparameters: {'bootstrap': True, 'max_depth': 20, 'max_features': 'log2', 'n_estimators': 300, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [11]:
hyper_params = {
    k: best_cfg[k] for k in [
        "n_estimators",
        "max_depth",
        "min_samples_split",
        "min_samples_leaf",
        "max_features",
        "bootstrap"
    ]
}

print("Hyperparameters:", hyper_params)

Hyperparameters: {'n_estimators': 300, 'max_depth': 20, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'bootstrap': True}


In [ ]:
model = RandomForestClassifier(
    **hyper_params,
    random_state=42,
    n_jobs=-1
)

# Train trên toàn bộ tập train
model.fit(x_train, y_train)

RandomForestClassifier(max_depth=20, max_features='log2', n_estimators=300,
                       n_jobs=-1, random_state=42)

In [20]:
y_pred = model.predict(x_val)

print(classification_report(y_val,y_pred))

              precision    recall  f1-score   support

    11222333       1.00      1.00      1.00        20
    11225430       1.00      1.00      1.00        20
    11226754       1.00      1.00      1.00        20
    11226767       1.00      1.00      1.00        18
    11226938       1.00      1.00      1.00        20

    accuracy                           1.00        98
   macro avg       1.00      1.00      1.00        98
weighted avg       1.00      1.00      1.00        98



In [17]:
# types and names of the artifacts
artifact_type = "inference_artifact"
artifact_model = "model_export"

In [18]:
logger.info("Dumping the artifacts to disk")
# Save the model using joblib
joblib.dump(model, artifact_model)

29-04-2025 23:34:51 Dumping the artifacts to disk


['model_export']

In [19]:
# Model artifact
artifact = wandb.Artifact(artifact_model,
                          type=artifact_type,
                          description="Best Random Forest model"
                          )

logger.info("Logging model artifact")
artifact.add_file(artifact_model)
run.log_artifact(artifact)

29-04-2025 23:36:00 Logging model artifact


<Artifact model_export>

In [22]:
run.finish()